### Import Libraries

In [1]:
# add path to custom python code for accessing data lake and working with dataframes
import sys
sys.path.append('/Users/markbills/Library/CloudStorage/OneDrive-Transformativ,LLC/Clients/Ovation Holdings/src')

# Azure Data Lake libraries
import azure_data_lake_interface as adl

# Helper function libraries
import helper_functions as hf

# Data analysis libraries
import weekly_margin_report as wmr

# display support
from IPython.display import display, Markdown

### Load Data

In [1]:
# attach to the data lake
config = hf.load_config("config/datalake_config.json", flush_cache=True)
service_client = adl.get_azure_service_client(config["blob_url"])
file_system_client = adl.get_azure_file_system_client(service_client, "consolidated")

# get data lake data
trans_type = "CustInvc"
end_date = "2025-05"
filename = f"monthly_{trans_type}_margin_analysis_dataset_2022-01_{end_date}.parquet"
combined_monthly_data = adl.get_parquet_file_from_data_lake(file_system_client, "presentation/margin_analysis", filename)

NameError: name 'hf' is not defined

### Examine Data

In [3]:
Markdown(wmr.describe_dataset(combined_monthly_data))

**Data Range**: 2022-01 to 2025-05<br>**Number of Inventory Items**: 51,753<br>**Fields in Dataset**:<br>&ensp;&ensp;&ensp;- avg_quoted_cost (number)<br>&ensp;&ensp;&ensp;- avg_unit_cost (number)<br>&ensp;&ensp;&ensp;- commission_avg_margin (number)<br>&ensp;&ensp;&ensp;- commission_avg_margin_pct (number)<br>&ensp;&ensp;&ensp;- commission_avg_unit_price (number)<br>&ensp;&ensp;&ensp;- description (text)<br>&ensp;&ensp;&ensp;- highest_recent_unit_cost (number)<br>&ensp;&ensp;&ensp;- inventory_avg_margin (number)<br>&ensp;&ensp;&ensp;- inventory_avg_margin_pct (number)<br>&ensp;&ensp;&ensp;- inventory_avg_unit_price (number)<br>&ensp;&ensp;&ensp;- item_name (text)<br>&ensp;&ensp;&ensp;- item_type (text)<br>&ensp;&ensp;&ensp;- level_1_category (text)<br>&ensp;&ensp;&ensp;- level_2_category (text)<br>&ensp;&ensp;&ensp;- level_3_category (text)<br>&ensp;&ensp;&ensp;- level_4_category (text)<br>&ensp;&ensp;&ensp;- level_5_category (text)<br>&ensp;&ensp;&ensp;- level_6_category (text)<br>&ensp;&ensp;&ensp;- manufacturer (text)<br>&ensp;&ensp;&ensp;- monthly_commission_qty (number)<br>&ensp;&ensp;&ensp;- monthly_commission_sales (number)<br>&ensp;&ensp;&ensp;- monthly_inventory_qty (number)<br>&ensp;&ensp;&ensp;- monthly_inventory_sales (number)<br>&ensp;&ensp;&ensp;- monthly_quantity_purchased (number)<br>&ensp;&ensp;&ensp;- monthly_spend (number)<br>

In [4]:
combined_monthly_data.head()

item_name  \
month   sku                                       
2022-01 30         911BDCM01A .5 x 1 KUNKLE SRV   
        31       910BGFM01A 1.25 x 2 KUNKLE SRV   
        37   171SD01 .75 KUNKLE SRV (Scrd. Cap)   
        44     910BEDM01A .75 x 1.25 KUNKLE SRV   
        47         910BDCM01A .5 x 1 KUNKLE SRV   

                                 description       item_type manufacturer  \
month   sku                                                                 
2022-01 30     911BDCM01  \r\n.5" KUNKLE SRV  Inventory Item       Kunkle   
        31   910BGFM01  \r\n1.25" KUNKLE SRV  Inventory Item       Kunkle   
        37      171S-D01 \r\n.75" KUNKLE SRV  Inventory Item       Kunkle   
        44    910BEDM01  \r\n.75" KUNKLE SRV  Inventory Item       Kunkle   
        47     910BDCM01  \r\n.5" KUNKLE SRV  Inventory Item       Kunkle   

            level_1_category level_2_category level_3_category  \
month   sku                                                      
2022-01 30            Valves         Pressure              PRV   
        31            Valves         Pressure              PRV   
        37            Valves         Pressure              PRV   
        44            Valves         Pressure              PRV   
        47            Valves         Pressure              PRV   

            level_4_category level_5_category level_6_category  ...  \
month   sku                                                     ...   
2022-01 30               900              0.5              New  ...   
        31               900             1.25              New  ...   
        37               171             0.75              New  ...   
        44               900             0.75              New  ...   
        47               900              0.5              New  ...   

             monthly_commission_sales  avg_unit_cost  \
month   sku                                            
2022-01 30                        0.0         817.95   
        31                        0.0         813.40   
        37                        0.0            NaN   
        44                        0.0            NaN   
        47                        0.0            NaN   

             monthly_quantity_purchased  monthly_spend  avg_quoted_cost  \
month   sku                                                               
2022-01 30                          2.0         1635.9       817.950000   
        31                          1.0          813.4       813.400000   
        37                          NaN            NaN       725.200000   
        44                          NaN            NaN       556.760000   
        47                          NaN            NaN       525.466667   

             highest_recent_unit_cost  inventory_avg_margin  \
month   sku                                                   
2022-01 30                     817.95            385.050000   
        31                     813.40            757.600000   
        37                     680.00            454.000000   
        44                     494.55            503.116667   
        47                     574.35            227.650000   

             inventory_avg_margin_pct  commission_avg_margin  \
month   sku                                                    
2022-01 30                  32.007481                    NaN   
        31                  48.224061                    NaN   
        37                  40.035273                    NaN   
        44                  50.429335                    NaN   
        47                  28.385287                    NaN   

             commission_avg_margin_pct  
month   sku                             
2022-01 30                         NaN  
        31                         NaN  
        37                         NaN  
        44                         NaN  
        47                         NaN  

[5 rows x 25 columns]

### Explore Data

In [6]:
subset = combined_monthly_data[['item_type', 'manufacturer', 'item_name', 'description', 'monthly_inventory_qty',
                                'monthly_inventory_sales', 'avg_quoted_cost', 'highest_recent_unit_cost']].copy()
subset.rename(columns={'monthly_inventory_qty': 'qty', 'monthly_inventory_sales': 'sales', 'avg_quoted_cost': 'quoted_cost',
                              'cost_difference': 'difference', 'highest_recent_unit_cost': 'max_cost'}, inplace=True)
subset.dropna(inplace=True)
subset.quoted_cost = round(subset.quoted_cost, 2)
subset = subset[subset['sales'] > 0]
subset['difference'] = subset['quoted_cost'] - subset['max_cost']
subset['pct_diff'] = round((subset.difference / subset['max_cost']) * 100, 2)
subset

item_type         manufacturer  \
month   sku                                            
2022-01 30       Inventory Item               Kunkle   
        31       Inventory Item               Kunkle   
        37       Inventory Item               Kunkle   
        44       Inventory Item               Kunkle   
        47       Inventory Item               Kunkle   
...                         ...                  ...   
2025-04 2841927  Inventory Item  Jbt Thorpe Machines   
        2847903  Inventory Item                 Asco   
        2849497  Inventory Item               Bettis   
        2929786  Inventory Item                Other   
        2935074  Inventory Item               Fisher   

                                          item_name  \
month   sku                                           
2022-01 30             911BDCM01A .5 x 1 KUNKLE SRV   
        31           910BGFM01A 1.25 x 2 KUNKLE SRV   
        37       171SD01 .75 KUNKLE SRV (Scrd. Cap)   
        44         910BEDM01A .75 x 1.25 KUNKLE SRV   
        47             910BDCM01A .5 x 1 KUNKLE SRV   
...                                             ...   
2025-04 2841927                             2841927   
        2847903                             2847903   
        2849497                    Shafer VA68108-F   
        2929786                             2929786   
        2935074                             2935074   

                                                       description  qty  \
month   sku                                                               
2022-01 30                           911BDCM01  \r\n.5" KUNKLE SRV  2.0   
        31                         910BGFM01  \r\n1.25" KUNKLE SRV  1.0   
        37                            171S-D01 \r\n.75" KUNKLE SRV  1.0   
        44                          910BEDM01  \r\n.75" KUNKLE SRV  3.0   
        47                           910BDCM01  \r\n.5" KUNKLE SRV  6.0   
...                                                            ...  ...   
2025-04 2841927          ASCO 8362A101H1832F1 24VDC Solenoid Valve  1.0   
        2847903  Parker (392097) 7321KBN3SV00N0H111C2\r\n3/8, 2...  1.0   
        2849497               Shafer VA68108-F Trigger Valve Assy.  2.0   
        2929786  11A5214X102  Valve Plug & Stem Assembly, for P...  1.0   
        2935074  TRIM SET, 2" FISHER ED/ET, 2-5/16" PORT, 316SS...  2.0   

                  sales  quoted_cost  max_cost  difference  pct_diff  
month   sku                                                           
2022-01 30       2406.0       817.95    817.95        0.00      0.00  
        31       1571.0       813.40    813.40        0.00      0.00  
        37       1134.0       725.20    680.00       45.20      6.65  
        44       2993.0       556.76    494.55       62.21     12.58  
        47       4812.0       525.47    574.35      -48.88     -8.51  
...                 ...          ...       ...         ...       ...  
2025-04 2841927   671.0       469.85    469.85        0.00      0.00  
        2847903   302.8       211.95    211.95        0.00      0.00  
        2849497  1930.0       745.50    745.50        0.00      0.00  
        2929786   460.0       298.00    298.00        0.00      0.00  
        2935074  4424.9      1604.00   1604.00        0.00      0.00  

[28716 rows x 10 columns]

In [7]:
subset_lines = subset.shape[0]
subset_max_price = subset.query('difference >= 0').shape[0]
print(f"Pct of items where average monthly quoted cost was greater than or equal to 12-month maximum cost: {(subset_max_price/subset_lines)*100:.2f}%")

Pct of items where average monthly quoted cost was greater than or equal to 12-month maximum cost: 57.66%


In [8]:
below_max = subset.query('difference < -1')
below_max = below_max[below_max['manufacturer'] != 'null']
ttm_below_max = below_max.loc[below_max.index.get_level_values('month') >= '2024-04'].copy()
ttm_below_max.describe().iloc[1:]

,qty,sales,quoted_cost,max_cost,difference,pct_diff
mean,3.878582,4800.267740,1021.930451,1284.465473,-262.535022,-22.181437
std,7.065687,11776.491995,1936.987810,2449.600569,715.592984,21.184692
min,1.000000,8.000000,0.000000,5.120000,-23475.000000,-100.000000
25%,1.000000,448.000000,134.910000,168.050000,-223.802500,-31.935000
50%,2.000000,1488.500000,339.870000,432.400000,-55.340000,-15.535000
75%,4.000000,4280.000000,1032.695000,1293.950000,-13.592500,-6.107500
max,128.000000,266928.740000,31800.000000,55275.000000,-1.010000,-0.090000


In [9]:
ttm_below_max

item_type  manufacturer  \
month   sku                                     
2024-04 18       Inventory Item        Kunkle   
        19       Inventory Item        Kunkle   
        20       Inventory Item        Kunkle   
        22       Inventory Item        Kunkle   
        25       Inventory Item        Kunkle   
...                         ...           ...   
2025-04 1952337  Inventory Item  Consolidated   
        2403740  Inventory Item        Farris   
        2488033  Inventory Item        Fisher   
        2591918  Inventory Item        Kimray   
        2651565  Inventory Item        Fisher   

                                                         item_name  \
month   sku                                                          
2024-04 18                                300LJG 1.5x2.5 (300x150)   
        19                         911BHGM01A 1.5 x 2.5 KUNKLE SRV   
        20                      910BJHM01A-LP 2x3 (200# and BELOW)   
        22                          911BGFM01A 1.25 x 2 KUNKLE SRV   
        25                                       171H 2 KUNKLE SRV   
...                                                            ...   
2025-04 1952337  BCKFLN01 LP NACE Bellows (4659828) 1905-06 D/E...   
        2403740                                          18200-649   
        2488033                                            2488033   
        2591918                                            2591918   
        2651565               Fisher® 846-DM1H1/MTG3-846-B2/F2G9C6   

                                                       description  qty  \
month   sku                                                               
2024-04 18                             300LJG01 \r\n1.5" KUNKLE SV  4.0   
        19                          911BHGM01  \r\n1.5" KUNKLE SRV  2.0   
        20       910BJHM01\r\n(LP design- 200# & Below)\r\n2" K...  1.0   
        22                         911BGFM01  \r\n1.25" KUNKLE SRV  3.0   
        25                               171-H01 \r\n2" KUNKLE SRV  1.0   
...                                                            ...  ...   
2025-04 1952337             BCKFLN01 \r\nLP NACE BELLOWS 1905-06 D  1.0   
        2403740                            18200-649 Kalrez O-Ring  3.0   
        2488033  2U223733272 \r\nCage, Equal Percentage, 17-4PH...  1.0   
        2591918        Kimray T2993  7/16" EP Trim Set for 2" HPMV  4.0   
        2651565  Fisher® 846-DM1H1/MTG3-846-B2/F2G9C6\r\nU0006\...  1.0   

                    sales  quoted_cost  max_cost  difference  pct_diff  
month   sku                                                             
2024-04 18       11608.00      2321.55   3483.00    -1161.45    -33.35  
        19        7550.00      1915.55   2723.40     -807.85    -29.66  
        20        2775.00      1690.21   2371.50     -681.29    -28.73  
        22        7528.00      1417.21   2192.40     -775.19    -35.36  
        25        2143.00      1235.60   1286.00      -50.40     -3.92  
...                   ...          ...       ...         ...       ...  
2025-04 1952337   3764.53      1996.17   2823.28     -827.11    -29.30  
        2403740   3924.00       850.00   2042.00    -1192.00    -58.37  
        2488033    480.00       310.00    817.00     -507.00    -62.06  
        2591918    723.28       107.67    150.16      -42.49    -28.30  
        2651565   1354.00       985.00   1015.00      -30.00     -2.96  

[4188 rows x 10 columns]

In [10]:
print(f"Total TTM Sales with Below Max Quoted Costs: ${ttm_below_max.sales.sum():,.2f}")
avg_target_margin = 0.3
print(f"Addition TTM Sales at {avg_target_margin*100:.0f}% average margin: ${-1 * (ttm_below_max.difference.sum()/(1-avg_target_margin)):,.2f}")

Total TTM Sales with Below Max Quoted Costs: $20,103,521.29
Addition TTM Sales at 30% average margin: $1,570,709.53


In [11]:
item_type = "Inventory Item"
manufacturer = "Consolidated"
below_mfr_subset = ttm_below_max[(ttm_below_max.manufacturer == manufacturer) & (ttm_below_max.item_type == item_type)]
below_mfr_subset.describe().iloc[1:]

,qty,sales,quoted_cost,max_cost,difference,pct_diff
mean,3.367826,6084.484709,1382.528191,1730.195700,-347.667508,-21.254383
std,6.160137,13291.575232,2323.455968,2849.983858,772.930826,18.798854
min,1.000000,10.000000,0.000000,5.880000,-14991.300000,-100.000000
25%,1.000000,611.500000,168.425000,218.657500,-330.190000,-29.632500
50%,2.000000,1989.500000,488.435000,628.105000,-86.910000,-16.280000
75%,3.000000,5760.000000,1519.830000,1885.890000,-16.932500,-7.697500
max,116.000000,266928.740000,24175.660000,24404.450000,-1.012000,-0.100000


In [12]:
below_mfr_subset

item_type  manufacturer  \
month   sku                                     
2024-04 126      Inventory Item  Consolidated   
        128      Inventory Item  Consolidated   
        129      Inventory Item  Consolidated   
        134      Inventory Item  Consolidated   
        143      Inventory Item  Consolidated   
...                         ...           ...   
2025-04 37978    Inventory Item  Consolidated   
        39037    Inventory Item  Consolidated   
        39372    Inventory Item  Consolidated   
        482126   Inventory Item  Consolidated   
        1952337  Inventory Item  Consolidated   

                                                         item_name  \
month   sku                                                          
2024-04 126                         GK38 GASKET KIT 1905/06/10/12H   
        128                              4845101 GUIDE FOR 1900Q-2   
        129                                               TDK1900K   
        134                        .75 19110LC-2-CC-MS-31-MT-FT-GS   
        143                                               1543H-21   
...                                                            ...   
2025-04 37978                                          PSGK-34E019   
        39037                               31032013 EPR 75 O-RING   
        39372                                              3650109   
        482126                                              482126   
        1952337  BCKFLN01 LP NACE Bellows (4659828) 1905-06 D/E...   

                                                 description   qty     sales  \
month   sku                                                                    
2024-04 126                                  GK38 Gasket Kit   6.0   2166.00   
        128                                    4845101 Guide   2.0  11227.00   
        129                          TDK1900K Thermodisc Kit   4.0   3640.00   
        134      19110LC-2-MS-MT-FT\r\n.75" CONSOLIDATED SRV  15.0  12763.91   
        143                 1543H-21\r\n1.5" CONSOLIDATED SV   1.0    603.00   
...                                                      ...   ...       ...   
2025-04 37978                      PSGK-34E019 KIT 39MV07/37   2.0   1548.00   
        39037                     31032013\r\n*EPR 75 O-RING   2.0    122.00   
        39372                    3650109 Bonnet/Guide Gasket   2.0    122.00   
        482126                         0284901 Spring Washer   4.0    358.00   
        1952337       BCKFLN01 \r\nLP NACE BELLOWS 1905-06 D   1.0   3764.53   

                 quoted_cost  max_cost  difference  pct_diff  
month   sku                                                   
2024-04 126           199.99    203.28       -3.29     -1.62  
        128          3447.38   4276.00     -828.62    -19.38  
        129           253.52    281.41      -27.89     -9.91  
        134           492.43    514.27      -21.84     -4.25  
        143           351.47    675.22     -323.75    -47.95  
...                      ...       ...         ...       ...  
2025-04 37978         210.90    521.19     -310.29    -59.53  
        39037           0.07     35.23      -35.16    -99.80  
        39372          33.08     36.52       -3.44     -9.42  
        482126         56.98     64.03       -7.05    -11.01  
        1952337      1996.17   2823.28     -827.11    -29.30  

[2300 rows x 10 columns]

In [13]:
print(f"Total TTM Sales with Below Max Quoted Costs for {manufacturer}: ${below_mfr_subset.sales.sum():,.2f}")
print(f"Addition TTM Sales from {manufacturer} at {avg_target_margin*100:.0f}% average margin: ${-1 * (below_mfr_subset.difference.sum()/(1-avg_target_margin)):,.2f}")

Total TTM Sales with Below Max Quoted Costs for Consolidated: $13,994,314.83
Addition TTM Sales from Consolidated at 30% average margin: $1,142,336.10


In [14]:
ttm_below_max.to_excel('data_tables/ttm_quoted_cost_below_twelve_month_max_cost.xlsx')

In [15]:
above_max = subset.query('difference > 0').copy()
above_max = above_max[above_max['manufacturer'] != 'null']
ttm_above_max = above_max.loc[above_max.index.get_level_values('month') >= '2024-04'].copy()
ttm_above_max.describe().iloc[1:]

/Users/markbills/Library/CloudStorage/OneDrive-Transformativ,LLC/Clients/Ovation Holdings/Phase 2/Operational Visibility Workstream/MarginAnalysis/.venv/lib/python3.12/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,qty,sales,quoted_cost,max_cost,difference,pct_diff
mean,2.839564,4874.530699,1686.814519,1464.759126,222.055393,inf
std,5.421794,9799.102740,3165.126585,2872.426558,761.877000,NaN
min,0.300000,2.920000,0.010000,0.000000,0.000800,0.0000
25%,1.000000,445.105000,168.655000,142.800000,6.977500,3.5100
50%,1.000000,1501.000000,510.660000,473.370000,30.705000,6.5700
75%,3.000000,5304.750000,1659.020000,1479.985000,131.670000,19.9325
max,84.000000,120668.000000,53695.140000,52470.790000,14000.000000,inf


In [18]:
ttm_above_max

item_type  manufacturer  \
month   sku                                     
2024-04 10891    Inventory Item         Watts   
        1165799  Inventory Item  Consolidated   
        12629    Inventory Item        Crosby   
        1333     Inventory Item        Kunkle   
        141      Inventory Item  Consolidated   
...                         ...           ...   
2025-04 2072841  Inventory Item     Norriseal   
        617998   Inventory Item  Consolidated   
        6701     Inventory Item  Consolidated   
        7236     Inventory Item  Consolidated   
        804104   Inventory Item      Fastenal   

                                                   item_name  \
month   sku                                                    
2024-04 10891    Inactivated + 174A .75 50# WATTS HW SECT IV   
        1165799                                      1165799   
        12629                                         200289   
        1333     6021EDT01A .75 KUNKLE SV  (Steam Use ONLY!)   
        141                                .75 2478D-1-31-DA   
...                                                      ...   
2025-04 2072841                                      2072841   
        617998                                        617998   
        6701                  4347701 DISC HOLDER-MONEL 1720   
        7236                 .75 19096LC-2-CC-MS-31-FT-FT-GS   
        804104                                        804104   

                                                       description   qty  \
month   sku                                                                
2024-04 10891       174A .\r\n75" WATTS SV\r\nSET @ 50 PSI HW {IV}   1.0   
        1165799                         LG434\r\nLEVER GROUP PLAIN   2.0   
        12629                             200289 Crosby Gasket Kit   3.0   
        1333     6021EDT01A \r\n.75" KUNKLE SV\r\n(TEFLON SEAT ...  15.0   
        141                 2478D-1-31-DA \r\n.75" CONSOLIDATED SV   7.0   
...                                                            ...   ...   
2025-04 2072841             SPRING PKG 0. 375/0. 500  316..2203016   1.0   
        617998                          LTR09 \r\nLiquid Trim Kit    1.0   
        6701                        4347701 DISC HOLDER-MONEL 1720   1.0   
        7236              19096LC-2-FT-FT\r\n.75" CONSOLIDATED SRV   1.0   
        804104   IK10-150 B7 Stud Bolt Kit – (12) 7/8”-9 X 4-3/...   3.0   

                   sales  quoted_cost  max_cost  difference  pct_diff  
month   sku                                                            
2024-04 10891     462.00       323.40    223.20      100.20     44.89  
        1165799  1184.00       552.09    504.00       48.09      9.54  
        12629    2411.00       599.33    587.00       12.33      2.10  
        1333     3840.00       150.42    145.60        4.82      3.31  
        141      2277.00       188.58    179.34        9.24      5.15  
...                  ...          ...       ...         ...       ...  
2025-04 2072841    26.25         8.12      7.14        0.98     13.73  
        617998   1618.42      1392.00    678.53      713.47    105.15  
        6701     5461.00      3907.13   3418.74      488.39     14.29  
        7236     1973.00      1301.92    580.06      721.86    124.45  
        804104    300.00        59.65     43.05       16.60     38.56  

[533 rows x 10 columns]

In [19]:
print(f"Total TTM Sales with Above Max Quoted Costs: ${ttm_above_max.sales.sum():,.2f}")
print(f"Lost TTM Sales at {avg_target_margin*100:.0f}% average margin: ${-1 * (ttm_above_max.difference.sum()/(1-avg_target_margin)):,.2f}")

Total TTM Sales with Above Max Quoted Costs: $2,790,467.70
Lost TTM Sales at 30% average margin: $-192,342.44


In [20]:
item_type = "Inventory Item"
manufacturer = "Consolidated"
above_mfr_subset = ttm_above_max[(ttm_above_max.manufacturer == manufacturer) & (ttm_above_max.item_type == item_type)]
above_mfr_subset.describe().iloc[1:]

,qty,sales,quoted_cost,max_cost,difference,pct_diff
mean,2.304348,9358.231691,3431.407198,2904.739889,526.667309,78.595411
std,2.732247,12861.334447,3860.998192,3361.459242,1447.865291,721.713450
min,1.000000,20.000000,5.500000,0.153100,0.010000,0.000000
25%,1.000000,1387.000000,447.940000,368.585000,15.185000,2.860000
50%,1.000000,5800.000000,2337.750000,1788.800000,104.860000,8.330000
75%,2.000000,12884.000000,4609.590000,3852.585000,490.640000,22.725000
max,23.000000,120668.000000,17332.900000,16618.450000,14000.000000,10311.500000


In [21]:
above_mfr_subset

item_type  manufacturer  \
month   sku                                     
2024-04 1165799  Inventory Item  Consolidated   
        141      Inventory Item  Consolidated   
        2060735  Inventory Item  Consolidated   
        265      Inventory Item  Consolidated   
        2803     Inventory Item  Consolidated   
...                         ...           ...   
2025-04 1736693  Inventory Item  Consolidated   
        193      Inventory Item  Consolidated   
        617998   Inventory Item  Consolidated   
        6701     Inventory Item  Consolidated   
        7236     Inventory Item  Consolidated   

                                       item_name  \
month   sku                                        
2024-04 1165799                          1165799   
        141                    .75 2478D-1-31-DA   
        2060735                          2060735   
        265                             1905HC-1   
        2803                            1905EC-2   
...                                          ...   
2025-04 1736693                          1736693   
        193      1.0 19110LC-2-CC-MS-31-MT-FT-GS   
        617998                            617998   
        6701      4347701 DISC HOLDER-MONEL 1720   
        7236     .75 19096LC-2-CC-MS-31-FT-FT-GS   

                                                       description  qty  \
month   sku                                                               
2024-04 1165799                         LG434\r\nLEVER GROUP PLAIN  2.0   
        141                 2478D-1-31-DA \r\n.75" CONSOLIDATED SV  7.0   
        2060735                        31022346\r\nO RING VITON 75  1.0   
        265                       1905HC \r\n1.5" CONSOLIDATED SRV  2.0   
        2803                       1905EC-2\r\n1" CONSOLIDATED SRV  1.0   
...                                                            ...  ...   
2025-04 1736693                                    4650702 Guide L  2.0   
        193      19110LC-2-CC-MS-31-MT-FT-GS\r\n1" CONSOLIDATED...  1.0   
        617998                          LTR09 \r\nLiquid Trim Kit   1.0   
        6701                        4347701 DISC HOLDER-MONEL 1720  1.0   
        7236              19096LC-2-FT-FT\r\n.75" CONSOLIDATED SRV  1.0   

                    sales  quoted_cost  max_cost  difference  pct_diff  
month   sku                                                             
2024-04 1165799   1184.00       552.09    504.00       48.09      9.54  
        141       2277.00       188.58    179.34        9.24      5.15  
        2060735     25.00        60.06     50.00       10.06     20.12  
        265      11425.00      3832.87   3728.51      104.36      2.80  
        2803      3788.00      2651.86   2526.69      125.17      4.95  
...                   ...          ...       ...         ...       ...  
2025-04 1736693   5900.00      2026.45   1788.80      237.65     13.29  
        193       1479.00       917.56    540.89      376.67     69.64  
        617998    1618.42      1392.00    678.53      713.47    105.15  
        6701      5461.00      3907.13   3418.74      488.39     14.29  
        7236      1973.00      1301.92    580.06      721.86    124.45  

[207 rows x 10 columns]

In [22]:
print(f"Total TTM Sales with Above Max Quoted Costs for {manufacturer}: ${above_mfr_subset.sales.sum():,.2f}")
print(f"Loss of TTM Sales from {manufacturer} at {avg_target_margin*100:.0f}% average margin: ${-1 * (above_mfr_subset.difference.sum()/(1-avg_target_margin)):,.2f}")

Total TTM Sales with Above Max Quoted Costs for Consolidated: $1,937,153.96
Loss of TTM Sales from Consolidated at 30% average margin: $-155,743.05
